<a href="https://colab.research.google.com/github/tjdux/CS/blob/main/04_2_OneHotEncoder%EC%99%80_ColumnTransformer_scikit_learn%EC%9C%BC%EB%A1%9C_%EB%B2%94%EC%A3%BC%ED%98%95_%EB%B3%80%EC%88%98_%EB%8B%A4%EB%A3%A8%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 노트북이 코랩에서 실행 중인지 체크합니다.
import os
import sys
if 'google.colab' in sys.modules:
    if not os.path.isdir('mglearn'):
        # mglearn을 다운받고 압축을 풉니다.
        !wget -q -O mglearn.tar.gz https://bit.ly/mglearn-tar-gz
        !tar -xzf mglearn.tar.gz
    !wget -q -O data.tar.gz https://bit.ly/data-tar-gz
    !tar -xzf data.tar.gz
    # 나눔 폰트를 설치합니다.
    !sudo apt-get -qq -y install fonts-nanum
    import matplotlib.font_manager as fm
    font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
    for fpath in font_files:
        fm.fontManager.addfont(fpath)

In [11]:
import sklearn
from preamble import *
import matplotlib

# 나눔 폰트를 사용합니다.
matplotlib.rc('font', family='NanumBarunGothic')
matplotlib.rcParams['axes.unicode_minus'] = False

/content/mglearn/datasets.py:31: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)


In [2]:
import pandas as pd

# 숫자 특성과 범주형 문자열 특성을 가진 DataFrame
demo_df = pd.DataFrame({"숫자 특성": [0, 1, 2, 1],
                        "범주형 특성": ["양말", "여우", "양말", "상자"]})
demo_df

,숫자 특성,범주형 특성
0,0,양말
1,1,여우
2,2,양말
3,1,상자


In [3]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False) # spare_output=False -> Numpy 배열 반환
print(ohe.fit_transform(demo_df))

[[1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]]


In [4]:
# 변환된 특성에 해당하는 원본 범주형 변수 이름
print(ohe.get_feature_names_out())

['숫자 특성_0' '숫자 특성_1' '숫자 특성_2' '범주형 특성_상자' '범주형 특성_양말' '범주형 특성_여우']


- `OneHotEncoder`: 모든 특성을 범주형이라고 가정
- `ColumnTransformer`
  - 입력 데이터에 있는 열마다 다른 변환 적용 가능
  - 연속형 특성과 범주형 특성에 다른 전처리 과정을 적용할때 매우 유용!

In [12]:
import os

data = pd.read_csv(
    os.path.join(mglearn.datasets.DATA_PATH, "adult.data"),
    header=None, index_col=False,
    names=["age", "workclass", "fnlwgt", "education", "education-num",
           "martial-status", "occupation", "relationship", "race", "gender",
           "capital-gain", "capital-loss", "hours-per-week", "native-country",
           "income"]
)
data = data[['age', 'workclass', 'education', 'gender', 'hours-per-week',
             'occupation', 'income']]
data.head()

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


- 원-핫-인코딩 + 연속형 변수에 스케일 조정 필요

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

ct = ColumnTransformer(
    [("scaling", StandardScaler(), ["age", "hours-per-week"]),
     ("onehot", OneHotEncoder(sparse_output=False),
      ["workclass", "education", "gender", "occupation"])]
)

In [14]:
from sklearn.model_selection import train_test_split

data_features = data.drop("income", axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    data_features, data.income, random_state=0
)

ct.fit(X_train)
X_train_trans = ct.transform(X_train)
print(X_train_trans.shape)

(24420, 44)


In [15]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_trans, y_train)

X_test_trans = ct.transform(X_test)
print(f"테스트 점수: {round(logreg.score(X_test_trans, y_test), 2)}")

테스트 점수: 0.81


In [16]:
# ColumnTransformer 안의 단계에 접근
ct.named_transformers_.onehot

OneHotEncoder(sparse_output=False)